In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v2 import CNN_v2

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v2((480, 640, 4), split_into_parts=40)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v2/weights/"

In [7]:
model_names = ["cnn_v2_40_1e_3_20",
               "cnn_v2_40_1e_5_20",
               "cnn_v2_40_1e_5_40",
               "cnn_v2_40_1e_5_01_0.75",
               "cnn_v2_40_1e_5_02_0.76",
               "cnn_v2_40_1e_5_03_0.80",
               "cnn_v2_40_1e_5_19_0.80"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [CNN_v2((480, 640, 4), split_into_parts=40) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6977 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6975 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6976 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6975 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6967 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 4s 4s/step - loss: 0.6974 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 4s 4s/step - loss: 0.6969 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model CNN/CNN_v2/weights/cnn_v2_40_1e_3_20 behaved:
For cut-of-point 0.9 had accuracy 35.81664922096994:
[-17 -29 -20 -21 -28 -18 -24 -22 -17 -15 -14 -13 -19 -12 -11 -10 -13 -18
 -18 -10 -13 -17 -21 -23 -15 -17 -14 -16 -14 -12 -17 -26 -23 -17 -15 -14
  -4 -11 -11 -13  -9 -16 -16 -12 -14 -12  -4 -23 -20 -10 -17 -15 -13 -16
 -20 -21 -22 -11 -27 -30 -24 -32 -28 -23 -20 -24 -32 -27 -40 -24 -25 -25
 -15 -17 -20 -14 -21 -22 -19 -20 -27 -19 -21 -16 -13 -16 -18 -15 -11  -9
 -15 -17 -17 -32 -32 -33 -34 -29 -28 -18 -14 -15 -11 -16 -18 -20 -24 -27
 -30 -21 -30 -21 -22 -22 -27 -25 -20 -18 -19 -19 -31 -25 -20 -26 -20 -22
 -14 -18 -19 -14 -17 -15 -22 -20 -17 -23 -24 -18 -19 -11 -14 -22 -26 -21
 -25 -22 -20 -16 -22 -22 -23 -24 -22 -22 -23 -21 -23 -26 -21 -23 -27 -25
 -26 -16 -22 -27 -16 -21 -19 -15 -19 -16 -20 -16 -14 -17 -15 -20 -18 -14
 -18 -23 -23 -26 -22 -24 -29 -13 -17 -11 -24 -18 -19 -23 -23 -19 -14 -13
 -22 -20 -15 -14 -19 -23 -11  -9 -13 -12 -15 -21 -20 -17 -21 -23 -22 -19
 -17 -15 -22 -16  -